In [ ]:
import sys
import importlib
import pandas as pd
import numpy as np
import plotly.express as px

sys.path.append('../lib')
import pandas_utils
import pesys_utils
import autograd_convenience
import utils

importlib.reload(pandas_utils)
importlib.reload(pesys_utils)
importlib.reload(autograd_convenience)
importlib.reload(utils)

utils.set_secrets_as_env()

In [ ]:
try:
    con_popp.close()
except:
    pass
con_popp = pesys_utils.open_popp_connection()
df_oms = pandas_utils.pandas_from_sql("../sql/popp/godskrevet.sql", con_popp, tuning=10000, lowercase=True)
con_popp.close()

In [ ]:
try:
    con_pen.close()
except:
    pass
con_pen = pesys_utils.open_pen_connection()
df_bo = pandas_utils.pandas_from_sql("../sql/popp/bostedsland.sql", con_pen, tuning=10000, lowercase=True)
con_pen.close()

In [ ]:
df_bo.fnr_fk.value_counts()

In [ ]:
df = df_oms.merge(df_bo, how='left', on="fnr_fk")

In [ ]:
df.fnr_fk.value_counts()

In [ ]:
def prikk_antall(x):
    if x < 6:
        return -1
    else:
        return x

def minus1_to_lessthan6(x):
    if x == -1:
        return '<6'
    else:
        return x
    
def anonymize_small_categories(row, n=5):
    if row.antall < n:
        row.bostedsland = "Andre land"
        #row.avtaletype = "Annet"
        #row.avtalekriterie = "Annet"
    return row
    
    
eos = {"Belgia", "Bulgaria", "Danmark", "Estland", "Finland", "Frankrike", "Hellas", "Irland", 
       "Island", "Italia", "Kroatia", "Kypros", "Latvia", "Liechtenstein", "Litauen", "Luxembourg", 
       "Malta", "Nederland", "Norge", "Polen", "Portugal", "Romania", "Slovakia", "Slovenia", "Spania", 
       "Sveits", "Sverige", "Tsjekkia", "Tyskland", "Ungarn", "Østerrike"}

df["eøs_flagg"] = df.apply(lambda row: int(row.bostedsland in eos), axis=1) #| (row.avtaletype == "Eøs-Avtalen/Nordisk Konvensjon")), axis=1)
df.fillna("Ukjent", inplace=True)

In [ ]:
df_land = df.groupby(["bostedsland"], dropna=False).versjon.count().reset_index(name="antall")
df_land["eøs_flagg"] = df_land.bostedsland.apply(lambda x: int((x in eos)))
df_land = df_land.apply(anonymize_small_categories, axis=1)
df_land = df_land.groupby(["bostedsland"], dropna=False, as_index=False).antall.sum()
df_land["eøs_flagg"] = df_land.bostedsland.apply(lambda x: int((x in eos)))
df_land = df_land.sort_values("antall", ascending=False).reset_index(drop=True)

df_land[df_land.bostedsland == "Andre land"]

In [ ]:
df_counts = df.bostedsland.value_counts().to_frame()

In [ ]:
def sum_small_categories(x, n=5):
    if df_counts.loc[x].item() < n:
        return "Andre land"
    else:
        return x

In [ ]:
df_sunburst = df[~(df.bostedsland == 'Ukjent')]
df_sunburst.loc[:,"bostedsland"] = df_sunburst.bostedsland.apply(sum_small_categories)

fig_sunburst = px.sunburst(df_sunburst, path=["eøs_flagg", "bostedsland"], color_discrete_sequence=px.colors.qualitative.Dark2, height=800)
fig_sunburst.show()

In [ ]:
df_land

In [ ]:
def hent_token(user, pw):
    uri = 'https://freg-token-provider-q4.nais.preprod.local/token/user'
    headers = {'username': user,
               'password': pw}
    
    r = requests.get(uri, headers=headers)
    token = json.loads(r.text)['idToken']
    return token


def hent_header(srv_usr, srv_pwd):
    secret = (base64.b64encode(f'{srv_usr}:{srv_pwd}'.encode('UTF-8'))).decode()
    r = requests.post("https://security-token-service.nais.adeo.no/rest/v1/sts/token?grant_type=client_credentials&scope=openid", headers={"accept": "application/json", "Authorization": f"Basic {secret}"}, data="")
    #print({"accept": "application/json", "Authorization": f"Basic {secret}"})
    r.raise_for_status()
    token = json.loads(r.content)['access_token']

    return {
        'Authorization': 'Bearer {}'.format(token),
        'Nav-Consumer-Token': 'Bearer {}'.format(token),
        'Tema': 'GEN',
        'Accept': '*/*',
        'Content-Type': 'application/json'
    }


def hent_aktorider(df, user, token):
    headers = hent_header(user, token)
    df_list = []
    for i in range(0,len(df), 100):
        fnr=list(df.iloc[i:i+100].fnr.astype(str))
        query = """{
              "query": "query($identer: [ID!]!, $grupper: [IdentGruppe!]) { hentIdenterBolk(identer: $identer, grupper: $grupper, historikk: false){ident identer {ident historisk gruppe}code}}",
              "variables": {
              "identer": """+ str(fnr).replace('\'', '"') +""",
              "grupper": ["AKTORID"]
            }
        }"""
        response = requests.post("https://pdl-api.intern.nav.no/graphql", headers=headers, data=query)
        # Sjekker statuskode og henter ny header hvis utgått token
        if json.loads(response.text)['data']==None:
            headers = hent_header()
            response = requests.post("https://pdl-api.intern.nav.no/graphql", headers=headers, data=query)

        df_list.append(pd.json_normalize(json.loads(response.content)['data']['hentIdenterBolk']))
    df = pd.concat(df_list)
    df["aktorid"] = df.identer.apply(lambda x: x[0]['ident'])
    df = df.drop(columns=["identer", "code"])
    return df